US Cordinates are : (West, South, East, North)

-124,29,-70,49

North East cordinates are: -82.9, 33.1, -76.5,39.5

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/PrecipitationNowcasting")
os.getcwd()

import pandas as pd
import numpy as np
import xarray as xr 
import torch
import keras
# from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau ,ModelCheckpoint ,TensorBoard
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot
from keras.models import load_model

In [ ]:
xr_Jan20 = xr.open_mfdataset('data-virginia/Jan20/*.nc4')
xr_Feb20 = xr.open_mfdataset('data-virginia/Feb20/*.nc4')
xr_Mar20 = xr.open_mfdataset('data-virginia/Mar20/*.nc4')
xr_Apr20 = xr.open_mfdataset('data-virginia/Apr20/*.nc4')
xr_May20 = xr.open_mfdataset('data-virginia/May20/*.nc4')
xr_Jun20 = xr.open_mfdataset('data-virginia/Jun20/*.nc4')
xr_Jul20 = xr.open_mfdataset('data-virginia/Jul20/*.nc4')
xr_Aug20 = xr.open_mfdataset('data-virginia/Aug20/*.nc4')
xr_Sep20 = xr.open_mfdataset('data-virginia/Sep20/*.nc4')
xr_Oct20 = xr.open_mfdataset('data-virginia/Oct20/*.nc4')
xr_Nov20 = xr.open_mfdataset('data-virginia/Nov20/*.nc4')
xr_Dec20 = xr.open_mfdataset('data-virginia/Dec20/*.nc4')

In [ ]:
xr_all = xr.merge([xr_Jan20,xr_Feb20,xr_Mar20,xr_Apr20,xr_May20,xr_Jun20,xr_Jul20,xr_Aug20,xr_Sep20,xr_Oct20,xr_Nov20,xr_Dec20])
xr_all.to_netcdf('IMERG_finalrun_VA_NC_SC_2020.nc4')

In [ ]:
xr_1 = xr.open_dataset('IMERG_finalrun_VA_NC_SC_2020.nc4')

In [ ]:
df = xr_1.to_dataframe().reset_index().round(2)

In [ ]:
print(df.shape)
df["time"]= pd.to_datetime(df.time)
df = df.sort_values(by=["time","lat","lon"]).reset_index(drop=True)
print(df.shape)

In [ ]:

print(df["time"].nunique())


# CGAN model and evaluation metrics

In [ ]:
# CGAN model taken from machine learning mastery

# https://machinelearningmastery.com/how-to-develop-a-pix2pix-gan-for-image-to-image-translation/

# define the discriminator model
def define_discriminator(input_shape=(64, 64, 4), target_shape =(64, 64, 1)):

	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=input_shape)
	# target image input
	in_target_image = Input(shape=target_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model

In [ ]:
# discriminator = define_discriminator()
# discriminator.summary()

In [ ]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g
 
# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g
 
 # define the standalone generator model
def define_generator(image_shape=(64,64,3)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model
	e1 = define_encoder_block(in_image, 64, batchnorm=False)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e5)
	b = Activation('relu')(b)
	# decoder model
	d3 = decoder_block(b, e5, 512)
	d4 = decoder_block(d3, e4, 512, dropout=False)
	d5 = decoder_block(d4, e3, 256, dropout=False)
	d6 = decoder_block(d5, e2, 128, dropout=False)
	d7 = decoder_block(d6, e1, 64, dropout=False)
	# output
	g = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model


In [ ]:
generator = define_generator(image_shape=(64, 64, 4))
generator.summary()

In [ ]:
def SSIMLoss(y_true, y_pred):
  SSI_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0, filter_size = 3))
  mae_loss = tf.reduce_mean(tf.abs(y_true - y_pred))
  mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
  total_loss = SSI_loss + mae_loss
  return total_loss

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy',  'mse'], optimizer=opt, loss_weights=[1, 100])
	return model

In [ ]:
# select a batch of random samples, returns images and target
def generate_real_samples(data, target, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = data, target
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [ ]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, data, target, leadtime, plot_path, model_path, n_samples=3):
  # select a sample of input images
  [X_realA, X_realB], _ = generate_real_samples(data, target, n_samples, 1)
  # generate a batch of fake samples
  X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
  # scale all pixels from [-1,1] to [0,1]
  X_realA = (X_realA + 1) / 2.0
  X_realB = (X_realB + 1) / 2.0
  X_fakeB = (X_fakeB + 1) / 2.0
  # plot real source images
  pyplot.figure(figsize = (8, 8))
  for i in range(n_samples):
    pyplot.subplot(3, n_samples, 1 + i)
    pyplot.title('Real input images')
    pyplot.axis('off')
    pyplot.imshow(X_realA[i][:,:,-1])
  # plot generated target image
  for i in range(n_samples): 
    pyplot.subplot(3, n_samples, 1 + n_samples + i)
    pyplot.title('Generated target images')
    pyplot.axis('off')
    pyplot.imshow(X_fakeB[i][:,:,-1])
  # plot real target image
  for i in range(n_samples):
    pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
    pyplot.title('Real target images') 
    pyplot.axis('off')
    pyplot.imshow(X_realB[i][:,:,-1])
  # save plot to file
  filename1 = '%s/plot_%06d_%s.png' % (plot_path, step+1, leadtime)
  pyplot.savefig(filename1)
  pyplot.close()
  # save the generator model
  filename2 = '%s/model_%06d.h5' % (model_path, step+1)
  g_model.save(filename2)
  print('>Saved: %s and %s' % (filename1, filename2))

In [ ]:
# train pix2pix model
def train_model(d_model, g_model, gan_model, data, target, n_epochs, n_batch,leadtime, plot_path, model_path):
  # print("hi")
  # determine the output square shape of the discriminator
  n_patch = d_model.output_shape[1]
  # print("hi again")
  # unpack dataset
  trainA, trainB = data, target
  # calculate the number of batches per training epoch
  bat_per_epo = int(len(trainA) / n_batch)
  # calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs
  print(n_steps)
  # manually enumerate epochs
  for i in range(n_steps):
    # select a batch of real samples
    [X_realA, X_realB], y_real = generate_real_samples(data, target, n_batch, n_patch)
    # generate a batch of fake samples
    X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
    # update discriminator for real samples
    d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
    # update discriminator for generated samples
    d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
    # update the generator
    g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
    # summarize performance
    print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    # summarize model performance
    if (i+1) % (bat_per_epo * 10) == 0:
      summarize_performance(i, g_model, data, target,leadtime, plot_path, model_path)


In [ ]:
def prep_clf(obs, sim, threshold=0.5):

    obs = np.where(obs >= threshold, 1, 0)
    sim = np.where(sim >= threshold, 1, 0)

    # True positive (TP)
    hits = np.sum((obs == 1) & (sim == 1))

    # False negative (FN)
    misses = np.sum((obs == 1) & (sim == 0))

    # False positive (FP)
    falsealarms = np.sum((obs == 0) & (sim == 1))

    # True negative (TN)
    correctnegatives = np.sum((obs == 0) & (sim == 0))

    return hits, misses, falsealarms, correctnegatives

def CSI(obs, sim, threshold=0.5):

    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)

    return hits / (hits + misses + falsealarms)

def FAR(obs, sim, threshold=0.5):
  
    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)

    return falsealarms / (hits + falsealarms)


def POD(obs, sim, threshold=0.5):
    
    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)

    return hits / (hits + misses)


def HSS(obs, sim, threshold=0.5):
    
    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)

    HSS_num = 2 * (hits * correctnegatives - misses * falsealarms)
    HSS_den = (misses**2 + falsealarms**2 + 2*hits*correctnegatives +
               (misses + falsealarms)*(hits + correctnegatives))

    return HSS_num / HSS_den


# Helper functions to derive dataset-A and dataset-B

In [ ]:
#Function to retrieve images with atleast minimum number of rainy pixels (Additional parameter threshold is to selected the minimum rainfall rate)
def get_sequences_gt_threshold(threshold, train, percentage_of_rain_pixels):
  selected_images = dict()
  indx = []
  pixels = train.shape[1]
  for i in range(0, len(train)): 
    img = train[i]
    pixels_gt = img[img > threshold]
    if len(pixels_gt) >= int(percentage_of_rain_pixels * pixels):
      selected_images[i] = img
      indx.append(i)
  # Breakes sequence into multiple sequences whenever there is gap in the index (idx)
  # and saves those sequences to list
  seq_50 = []
  all_50 =[]
  for i in range(0, len(indx)-1):
    if indx[i+1] - indx[i] == 1:
      seq_50.append(indx[i])
    else:
      seq_50.append(indx[i])
      if len(seq_50) >= 3:      # sequence should be continuous with atleast three values
        all_50.append(seq_50)
      seq_50 = []
  # print(all_50)
  
  # Compares above generated sequences with key of dictionary (which has index and values as key-value pairs) 
  # and generates dataset for later use (for temporal windows)
  
  sub = []
  full_50 = []
  for i in all_50:
    for j in i:
      for k, v in selected_images.items():
        if j==k:
          sub.append(v)
    full_50.append(sub)
    sub = []
  return full_50



In [ ]:
def windowed_data(dataset, input_length, output_length):
  data = []
  labels = []
  for i in range(len(dataset)):
    single = dataset[i]
    for j in range(len(single) - input_length-output_length + 1):
      ind = single[j: j+input_length]
      data.append(ind)
      label = single[j+input_length+output_length-1]
      labels.append(label)
  return np.array(data), np.array(labels)


# Modelling on dataset-A using CGAN

In [ ]:
# full data
# Full data
X_full = df[["precipitationCal"]]

newdata = X_full.values
print(len(newdata))

full = np.array_split(newdata, 17548)
full = np.array(full)
print(full.shape)

full_50 = get_sequences_gt_threshold(0, full, 0.5) 
print(len(full_50))

print("=============== Full dataset ===================")


x_full_50, y_full_50 = windowed_data(full_50, 4, 8)

print(x_full_50.shape)
print(y_full_50.shape)

x_full_50 = x_full_50.transpose(0, 2, 1, 3)
print(x_full_50.shape)

X_full_50 = x_full_50.reshape(x_full_50.shape[0], 64, 64, x_full_50.shape[2] * x_full_50.shape[3])
Y_full_50 = y_full_50.reshape(y_full_50.shape[0], 64, 64, y_full_50.shape[2])
print(X_full_50.shape)
print(Y_full_50.shape)

print(X_full_50.max())
print(Y_full_50.max())


print(X_full_50.min())
print(Y_full_50.min())


full_50 = X_full_50.max()
X_full_50 = (X_full_50 - 0.5 * full_50) / (0.5 * full_50)
Y_full_50 = (Y_full_50 - 0.5 * full_50) / (0.5 * full_50)

print(X_full_50.max())
print(Y_full_50.max())


print(X_full_50.min())
print(Y_full_50.min())

In [ ]:
from sklearn.model_selection import KFold
# prepare cross validation
kfold = KFold(5)
i = 1
accuracy = []
CSI_all = []
POD_all = []
FAR_all = []
HSS_all = []

# enumerate splits
for train, test in kfold.split(X_full_50, Y_full_50):
  print(i) 
  print(X_full_50[train].shape, Y_full_50[train].shape, X_full_50[test].shape, Y_full_50[test].shape)
  data, target = X_full_50[train], Y_full_50[train]
  print('Loaded', data.shape, target.shape)
  input_shape = data.shape[1:]
  target_shape = target.shape[1:]
  # define the models
  d_model = define_discriminator(input_shape, target_shape)
  g_model = define_generator(input_shape)
  # define the composite model
  gan_model = define_gan(g_model, d_model, input_shape)
  # train model
  model = train_model(d_model, g_model, gan_model, data, target, 1, 16,'2hr', 'Plots_50','Models_50')
  # save the generator model
  filename2 = 'Model_50/model_%02d.h5' % i
  g_model.save(filename2)
  model = load_model(filename2)
  gen_images_20 = model.predict(X_full_50[test])
  SSI = tf.reduce_mean(tf.image.ssim(Y_full_50[test], gen_images_20, 1.0, filter_size = 3))
  accuracy.append(SSI)
  Y_pred_20 = (gen_images_20 * 0.5 * full_50) + (0.5 * full_50)
  Y_full_50[test]  = (Y_full_50[test] * 0.5 * full_50) + (0.5 * full_50)
  CSI_index = CSI(Y_full_50[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  POD_index = POD(Y_full_50[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  FAR_index = FAR(Y_full_50[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  HSS_index = HSS(Y_full_50[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  CSI_all.append(CSI_index)
  POD_all.append(POD_index)
  FAR_all.append(FAR_index)
  HSS_all.append(HSS_index)
  Y_full_50[test] = (Y_full_50[test] - 0.5 * full_50) / (0.5 * full_50)
  i = i + 1
  

In [ ]:
SSI_mean = tf.math.reduce_mean(accuracy)
print(SSI_mean)
# print(tf.math.reduce_mean(accuracy))
print(average(CSI_all))
print(average(POD_all))
print(average(FAR_all))
print(average(HSS_all))

# Modelling on dataset-B using CGAN

In [ ]:
# full data
# Full data
X_full = df[["precipitationCal"]]

newdata = X_full.values
print(len(newdata))

full = np.array_split(newdata, 17548)
full = np.array(full)
print(full.shape)

full_20 = get_sequences_gt_threshold(0, full, 0.2) 
print(len(full_20))

print("=============== Full dataset ===================")


x_full_20, y_full_20 = windowed_data(full_20, 4, 4)

print(x_full_20.shape)
print(y_full_20.shape)

x_full_20 = x_full_20.transpose(0, 2, 1, 3)
print(x_full_20.shape)

X_full_20 = x_full_20.reshape(x_full_20.shape[0], 64, 64, x_full_20.shape[2] * x_full_20.shape[3])
Y_full_20 = y_full_20.reshape(y_full_20.shape[0], 64, 64, y_full_20.shape[2])
print(X_full_20.shape)
print(Y_full_20.shape)

print(X_full_20.max())
print(Y_full_20.max())


print(X_full_20.min())
print(Y_full_20.min())


full_20 = Y_full_20.max()
X_full_20 = (X_full_20 - 0.5 * full_20) / (0.5 * full_20)
Y_full_20 = (Y_full_20 - 0.5 * full_20) / (0.5 * full_20)

print(X_full_20.max())
print(Y_full_20.max())


print(X_full_20.min())
print(Y_full_20.min())

In [ ]:
from sklearn.model_selection import KFold
# prepare cross validation
kfold = KFold(5)
i = 1
accuracy = []
CSI_all = []
POD_all = []
FAR_all = []
HSS_all = []

# enumerate splits
for train, test in kfold.split(X_full_20, Y_full_20):
  print(i) 
  print(X_full_20[train].shape, Y_full_20[train].shape, X_full_20[test].shape, Y_full_20[test].shape)
  data, target = X_full_20[train], Y_full_20[train]
  print('Loaded', data.shape, target.shape)
  input_shape = data.shape[1:]
  target_shape = target.shape[1:]
  # define the models
  d_model = define_discriminator(input_shape, target_shape)
  g_model = define_generator(input_shape)
  # define the composite model
  gan_model = define_gan(g_model, d_model, input_shape)
  # train model
  model = train_model(d_model, g_model, gan_model, data, target, 1, 64,'30mins', 'Plots_20','Models_20')
  # save the generator model
  filename2 = 'Model_20/model_%02d.h5' % i
  g_model.save(filename2)
  model = load_model(filename2)
  gen_images_20 = model.predict(X_full_20[test])
  SSI = tf.reduce_mean(tf.image.ssim(Y_full_20[test], gen_images_20, 1.0, filter_size = 3))
  accuracy.append(SSI)
  Y_pred_20 = (gen_images_20 * 0.5 * full_20) + (0.5 * full_20)
  Y_full_20[test]  = (Y_full_20[test] * 0.5 * full_20) + (0.5 * full_20)
  CSI_index = CSI(Y_full_20[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  POD_index = POD(Y_full_20[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  FAR_index = FAR(Y_full_20[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  HSS_index = HSS(Y_full_20[test].squeeze(), Y_pred_20.squeeze() , threshold = 0.5)
  CSI_all.append(CSI_index)
  POD_all.append(POD_index)
  FAR_all.append(FAR_index)
  HSS_all.append(HSS_index)
  Y_full_20[test] = (Y_full_20[test] - 0.5 * full_20) / (0.5 * full_20)
  i = i + 1
  

In [ ]:
SSI_mean = tf.math.reduce_mean(accuracy)
print(SSI_mean)
# print(tf.math.reduce_mean(accuracy))
print(average(CSI_all))
print(average(POD_all))
print(average(FAR_all))
print(average(HSS_all))